# The notebook displays the performance of damped Newton when using different types of distance functions. Here we also look into the distribution of eigenvectors of the Hessian over a two dimensional plane using a heatmap which helps in observing pattern of their distribution over the spectrum of eigenvalues.

In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs(relative_path_to_new_folder, exist_ok = True)
if not os.path.isdir('../Images/Eigenvectorstability_analysis_images'):
    os.makedirs('../Images/Eigenvectorstability_analysis_images')

In [ ]:
"""Different norms"""
def p_norms ( x, y, p ): 
   # L-1 norm
   if p == 1:
      return  abs( np.sum( x, 0 )[:,None] - np.sum( y, 0 )[None,:] )  
   # L-2 norm
   elif p == 2:
      return  np.sum( x**2, 0 )[:,None] + np.sum( y**2, 0 )[None,:] - 2 * x.transpose().dot( y )
   # L-4 norm
   elif p == 4:
    return np.sum( x**4, 0 )[:,None] + np.sum( y**4, 0 )[None,:] + 4 * ( x**3 ).transpose().dot( y ) - 4 * x.transpose().dot( y**3 ) + 6 * ( x**2 ).transpose().dot( y**2 )
   # L-infinity norm
   elif p == 'inf':
      return np.maximum( abs( x[0,:][:,None] - y[0,:][None,:] ), abs( x[1,:][:,None] - y[1:][None:] ) )


"""To compute distance matrix"""
def distmat( x, y ):
    return np.sum( x**2, 0 )[:,None] + np.sum( y**2, 0 )[None,:] - 2 * x.transpose().dot( y )

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP( u, K, v ):
    return u[:,None] * K * v[None,:]

def plotp( x, col, plt, scale = 200, edgecolors = "k"):
  return plt.scatter( x[0,:], x[1,:], s = scale, edgecolors = edgecolors, c = col, cmap = 'plasma', linewidths = 2 )

In [ ]:
N = [ 400, 500 ]

In [ ]:
import computational_OT


## Damped Newton algorithm:
The dual formulation of OT is given by
$$
OT_{\varepsilon} = \max_{f\in \mathbb{R}^{n}, g\in\mathbb{R}^{m}} \langle f, \alpha \rangle + \langle g, \beta \rangle - \varepsilon\left(\langle\alpha \otimes \beta, e^{\frac{f}{\varepsilon}}\odot K \odot e^{\frac{g}{\varepsilon}}  \rangle-1\right)\ ,
$$
where
$$
\alpha \in \mathcal{M}_{1}(\mathcal{X}),\ \beta \in \mathcal{M}_{1}(\mathcal{Y}),\ \varepsilon>0,\ f\in\mathbb{R}^{n},\ g\in \mathbb{R}^{m}\ .
$$

The Hessian is given by 
$\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)=\frac{-1}{\varepsilon}
\begin{pmatrix}
\Delta(\alpha) && \pi_{\varepsilon}\\
\pi^{T}_{\varepsilon} && \Delta(\beta) 
\end{pmatrix}
\ , \ $ where $\pi\mathbb{1}_{m} = \alpha,\ \pi^{T}\mathbb{1}_{n}=\beta,\ $ and $\Delta = diag: \mathbb{R}^{n} \rightarrow M_{n}(\mathbb{R})$ is the linear operator mapping a vector  to a diagonal matrix  containing  this vector.

This implies 
$$
\begin{pmatrix}
\Delta(\alpha) && \pi_{\varepsilon}\\
\pi^{T}_{\varepsilon} && \Delta(\beta) 
\end{pmatrix}
\begin{pmatrix}
\mathbb{1}_{n}\\
\mathbb{1}_{m}
\end{pmatrix} = 0\ ,
$$
that is,
$$
\begin{pmatrix}
\mathbb{1}_{n}\\
\mathbb{1}_{m}
\end{pmatrix}\in \ker(\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g))\ .
$$
Hence, $\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)$ is singular. Therefore, on regularization we have the following Hessian
$
H_{reg} := \nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)+\lambda cc^{T}\ ,
$ 
where $c= \begin{pmatrix}\frac{\mathbb{1}}{\sqrt{n+m}}\\-\frac{\mathbb{1}}{\sqrt{n+m}}\end{pmatrix}\in M_{(n+m),1}(\mathbb{R})$.

Now, at the $k^{th}$ iteration solve
$\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)p_{k} = \nabla Q_{\alpha, \beta,\varepsilon}(f,g)$ to obtain the optimizing direction vector $p_{k}$ and then perform the Armijo condition to obtain the update step $\alpha_{k}$ such that we have the update
$$
(f,g) \leftarrow (f,g) + \alpha_{k} p_{k}\ .
$$


In [ ]:
x = np.random.rand( 2, N[0] ) -  0.5
theta = 2 * np.pi * np.random.rand( 1, N[1] )
r = 0.8 + .2 * np.random.rand( 1, N[1] )
y = np.vstack( ( r * np.cos(theta), r * np.sin(theta) ) )

## L2-norm

In [ ]:
# Damped Newton
print("Damped Newton... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
dampedNewtonP_L2 = []
results_dampedNewton_L2  = []
times_dampedNewton_L2    = []
Hessians_dampedNewton_L2 = []
#Cost matrix
C = p_norms( x, y, 2 )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
epsilons = [ 1.0, 0.5, 0.1, 0.05, 0.03 ]
for eps in epsilons:
    print("For epsilon = "+str(eps)+":")    
    #Kernel
    K = np.exp( - C/eps )
    f, g = 0 * a, 0 * b
    print( " |- Iterating")  
    start = time.time()
    Optimizer = computational_OT.damped_Newton( K,
                                                a,
                                                b,
                                                f,
                                                g,
                                                eps,
                                                rho,
                                                c )
    out = Optimizer._update(    max_iterations = 50,
                                debug = False )
    end = time.time()
    if out != -1:
        results_dampedNewton_L2.append( out )
        times_dampedNewton_L2.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        dampedNewtonP_L2.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n" )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        Hessians_dampedNewton_L2.append( mat )
    else:
        epsilons.remove( eps )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewton_L2)):
  error = np.asarray( results_dampedNewton_L2[i]['error_a'] ) + np.asarray( results_dampedNewton_L2[i]['error_b'] )
  plt.plot( error,label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/ErrorLinesearchNewton.pdf", format = 'pdf')
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

### Objective function plot

In [ ]:
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( "Objective Function" )
for i in range(len(results_dampedNewton_L2)):
  value = np.asarray( results_dampedNewton_L2[i]['objective_values'] )
  plt.plot( value, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2)
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.yscale( 'log' )
plt.legend()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/ObjectiveLineSearchNewton.pdf", format = 'pdf' )
plt.show()

### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Alpha" )
for i in range(len(results_dampedNewton_L2)):
  plt.plot( np.asarray( results_dampedNewton_L2[i]["linesearch_steps"] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/AlphaLineSearchNewton.pdf", format = 'pdf' )
plt.show()

In [ ]:
def print_spectral_statistics( mat, stabilize = False ):
    if stabilize:
        # Stabilizing largest and smallest eigenvalue
        min_vector = np.hstack( ( np.ones( N[0] ), - np.ones( N[1] ) ) )
        max_vector = np.hstack( ( np.ones( N[0] ),  np.ones( N[1] ) ) )
        norm = np.sqrt( N[0] + N[1] )
        min_vector = min_vector/norm
        max_vector = max_vector/norm
        min_vector = min_vector.reshape( ( min_vector.shape[0], 1 ) )
        max_vector = max_vector.reshape( ( max_vector.shape[0], 1 ) )
        #
        mat = mat + np.dot( min_vector, min_vector.T )
        mat = mat - np.dot( max_vector, max_vector.T )
    # endif
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort( eig )
    eig = eig[ sorting_indices ]
    v   = v[ : , sorting_indices ]
    
    #print( "Mean eigenvalue: ", np.mean(eig) )
    print( "List of smallest eigenvalues: ", eig[ : 10 ] )
    print( "List of largest  eigenvalues: ", eig[ - 10 : ] )
    min_index = np.argmin( eig )
    max_index = np.argmax( eig )
    min_value = eig[ min_index ]
    max_value = eig[ max_index ]
    min_vector = v[ : , min_index]
    min_vector = min_vector/min_vector[0]
    max_vector = v[ : , max_index ]
    max_vector = max_vector/max_vector[0]
    condition_number = max_value/min_value
    # Test smallest and largest
    # print( "Min eigenvalue vector: ", min_vector)
    # print( "Max eigenvalue vector: ", max_vector)
    #
    #print( v[:,0]*np.sqrt( self.N1 + self.N2))
    #vector = v[:,0]
    #test = np.dot( result, vector)
    #print( np.linalg.norm(test) )
    #print("Min absolute eigenvalues: ", min_value)
    #print("Norm of v-1: ", np.linalg.norm(min_vector-eig_vector))
    print( "Condition number: ", condition_number )
    # plt.hist( eig, 50)
    # plt.title( "Histogram of eigenvalues for Hessian")
    # plt.xlabel( "Eigenvalues")
    # plt.yscale( "log" )
    # plt.show()
    return eig, v

In [ ]:
eigs = []
eigvecs = []
for i in range(len(epsilons)):
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    Hessian = Hessians_dampedNewton_L2[i]
    ev = print_spectral_statistics( Hessian, stabilize = False )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print( "" )
# end for

In [ ]:
fig, ax = plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title( "Histogram of eigenvalues." )
for i in range(len(epsilons)):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
    ax[i].set_xlim( 0, 2 )
# end for
plt.subplots_adjust( wspace = 0, hspace = 0 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/eigenhistunstabilized.pdf", format = 'pdf' )
plt.show()

### Heatmaps


#### Heatmaps of eigenvectors corresponding to the 4 smallest eigenvalues

In [ ]:
eigenmodes_count = 4
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:], y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = len( epsilons ), ncols = eigenmodes_count, sharey = True )
scale = 30
courant_modes = []
for i in range(len(epsilons)):
    eigenvectors = eigvecs[i]
    courant_modes.append( [] )
    for j in range(eigenmodes_count):
        function_as_vector = eigenvectors[:,j]
        function_as_vector = function_as_vector/np.max( function_as_vector )
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i-1][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
            #print( "epsilon: ", i, " -- corr = ", correlation )
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im1 = ax[i][j].scatter( X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ), c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im1, ax = ax[i][j] )
    # end for
    ax[i][0].set_title( "For smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][1].set_title( "For $2^{nd}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][2].set_title( "For $3^{rd}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][3].set_title( "For $4^{th}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    #ax[i][4].set_title("For $5^{th}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
# end for
#fig.colorbar(im1, ax=ax.ravel().tolist())
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:]) + .1 )
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/Eigenheatmaplowest4L2.pdf", format = 'pdf' )
plt.show();

#### Heatmaps of eigenvectors corresponding to the 16 smallest eigenvalues

In [ ]:
eigenmodes_count = 16
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:], y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = eigenmodes_count//4, ncols = eigenmodes_count//4, sharey = True )
scale = 30
courant_modes = []
k = 0
for i in range(eigenmodes_count//4):
    eigenvectors = eigvecs[1]
    courant_modes.append( [] )
    for j in range(eigenmodes_count//4):
        function_as_vector = eigenvectors[ : , j + 4 * k ]
        function_as_vector = function_as_vector/np.max( function_as_vector )
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i-1][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im1 = ax[i][j].scatter( X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ), c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im1, ax = ax[i][j] )
    # end for
    ax[i][0].set_title( "For "+str(1 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][1].set_title( "For "+str(2 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][2].set_title( "For "+str(3 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][3].set_title( "For "+str(4 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    #ax[i][4].set_title("For $5^{th}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
    k += 1
# end for
#fig.colorbar(im1, ax=ax.ravel().tolist())
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/EigenheatmaplowestL2.pdf", format = 'pdf' )
plt.show();

#### Heatmaps of eigenvectors corresponding to the 4 largest eigenvalues

In [ ]:
eigenmodes_count = 4
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( (x[1,:], y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = len(epsilons), ncols = eigenmodes_count, sharey = True )
scale = 30
courant_modes = []
for i in range(len(epsilons)):
    eigenvectors = eigvecs[i]
    courant_modes.append( [] )
    for j in range(eigenmodes_count):
        function_as_vector = eigenvectors[ :,N[0] + N[1] - j - 1 ]
        function_as_vector = function_as_vector/np.max( function_as_vector ) 
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i-1][j]
            correlation = np.dot( old_mode, function_as_vector )/(np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
            #print( "epsilon: ", i, " -- corr = ", correlation )
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im = ax[i][eigenmodes_count-j-1].scatter( X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ), c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im, ax = ax[i][ eigenmodes_count - j - 1 ])
    # end for
    ax[i][3].set_title( "For largest eigenvalue $\epsilon$ = " +str(epsilons[i]) )
    ax[i][2].set_title( "For $2^{nd}$ largest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][1].set_title( "For $3^{rd}$ largest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][0].set_title( "For $4^{th}$ largest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    #ax[i][0].set_title("For $5^{th}$ largest eigenvalue $\epsilon$= "+str(epsilons[i]))
# plt.axis("off")
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )   
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/Eigenheatmaplargest4L2.pdf", format = 'pdf' )
plt.show();

#### Heatmaps of eigenvectors corresponding to the 16 largest eigenvalues

In [ ]:
eigenmodes_count = 16
X = np.hstack( ( np.hstack( ( x[ 0 , : ], y[ 0 , : ] ) )[:,None], np.hstack( ( x[ 1 , : ], y[ 1, : ]) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = eigenmodes_count//4, ncols = eigenmodes_count//4, sharey = True )
scale = 30
courant_modes = []
k = 0
for i in range(eigenmodes_count//4):
    eigenvectors = eigvecs[ 1 ]
    courant_modes.append( [] )
    for j in range(eigenmodes_count//4):
        function_as_vector = eigenvectors[ : , N[0] + N[1] - j - 1 - 4 * k ]
        function_as_vector = function_as_vector/np.max(function_as_vector)
        # Flip sign if anti-correlated
        if i > 0 :
            old_mode = courant_modes[ i-1 ][ j ]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1 :
                function_as_vector = - function_as_vector
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im = ax[ i ][ eigenmodes_count//4 - j - 1 ].scatter( X.T[ 0, : ], X.T[ 1, : ], s = scale, edgecolors = ( 0, 0, 0, 0 ),  c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im, ax = ax[ i ][ eigenmodes_count//4 - j - 1 ])
    # end for
    ax[i][3].set_title( "For "+str(N[0] + N[1] - 1 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][2].set_title( "For "+str(N[0] + N[1] - 2 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][1].set_title( "For "+str(N[0] + N[1] - 3 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][0].set_title( "For "+str(N[0] + N[1] - 4 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    #ax[i][0].set_title("For $5^{th}$ largest eigenvalue $\epsilon$= "+str(epsilons[i]))
    k += 1
# plt.axis("off")
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )   
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout() 
plt.savefig( "../Images/Eigenvectorstability_analysis_images/EigenheatmaplargestL2.pdf", format = 'pdf' )
plt.show();

## L1-norm

In [ ]:
# Damped Newton
print("Damped Newton... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
dampedNewtonP_L1 = []
results_dampedNewton_L1  = []
times_dampedNewton_L1   = []
Hessians_dampedNewton_L1 = []
#Cost matrix
C = p_norms( x, y, 1 )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
epsilons = [ 1.0, 0.5, 0.1, 0.05, 0.03 ]
for eps in epsilons:
    print("For epsilon = "+str(eps)+":")       
    #Kernel
    K = np.exp( - C/eps )
    f, g = 0 * a, 0 * b
    print( " |- Iterating")  
    start = time.time()
    Optimizer = computational_OT.damped_Newton( K,
                                                a,
                                                b,
                                                f,
                                                g,
                                                eps,
                                                rho,
                                                c )
    out = Optimizer._update(    max_iterations = 50,
                                debug = False )
    end = time.time()
    if out != -1:
        results_dampedNewton_L1.append( out )
        times_dampedNewton_L1.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        dampedNewtonP_L1.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n" )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        Hessians_dampedNewton_L1.append( mat )
    else:
        epsilons.remove( eps )
# end for


### Error plot

In [ ]:
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewton_L1)):
  error = np.asarray( results_dampedNewton_L1[i]['error_a'] ) + np.asarray( results_dampedNewton_L1[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/ErrorLinesearchNewtonL1.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" ) 

### Objective function plot

In [ ]:
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( "Objective Function" )
for i in range(len(results_dampedNewton_L1)):
  value = np.asarray(results_dampedNewton_L1[i]['objective_values'])
  plt.plot( value, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.yscale( 'log' )
plt.legend()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/ObjectiveLineSearchNewtonL1.pdf", format = 'pdf' )
plt.show()


### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Alpha" )
for i in range(len(results_dampedNewton_L1)):
  plt.plot( np.asarray( results_dampedNewton_L1[i]["linesearch_steps"] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/AlphaLineSearchNewtonL1.pdf", format = 'pdf' )
plt.show()

In [ ]:
eigs = []
eigvecs = []
for i in range(len(epsilons)):
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    Hessian = Hessians_dampedNewton_L1[i]
    ev=print_spectral_statistics( Hessian, stabilize = False )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
fig, ax = plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title( "Histogram of eigenvalues." )
for i in range(len(epsilons)):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
    ax[i].set_xlabel("Eigenvalues")
    ax[i].set_yscale( "log" )
    ax[i].set_xlim( 0, 2 )
# end for
plt.subplots_adjust( wspace = 0, hspace = 0 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/eigenhistunstabilizedL1.pdf", format = 'pdf' )
plt.show()

### Heatmaps

#### Heatmaps of eigenvectors corresponding to the 4 smallest eigenvalues 

In [ ]:
eigenmodes_count = 4
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:], y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = len(epsilons), ncols = eigenmodes_count, sharey = True )
scale = 30
courant_modes = []
for i in range(len(epsilons)):
    eigenvectors = eigvecs[i]
    courant_modes.append( [] )
    for j in range(eigenmodes_count):
        function_as_vector = eigenvectors[:,j]
        function_as_vector = function_as_vector/np.max(function_as_vector)
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i-1][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
            #print( "epsilon: ", i, " -- corr = ", correlation )
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im1 = ax[i][j].scatter( X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ),  c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im1, ax = ax[i][j] )
    # end for
    ax[i][0].set_title( "For smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][1].set_title( "For $2^{nd}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][2].set_title( "For $3^{rd}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][3].set_title( "For $4^{th}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    #ax[i][4].set_title("For $5^{th}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
# end for
#fig.colorbar(im1, ax=ax.ravel().tolist())
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/Eigenheatmaplowest4L1.pdf", format = 'pdf' )
plt.show();

#### Heatmaps of eigenvectors corresponding to the 16 smallest eigenvalues 

In [ ]:
eigenmodes_count = 16
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:], y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = eigenmodes_count//4, ncols = eigenmodes_count//4, sharey = True )
scale = 30
courant_modes = []
k=0
for i in range(eigenmodes_count//4):
    eigenvectors = eigvecs[1]
    courant_modes.append( [] )
    for j in range(eigenmodes_count//4):
        function_as_vector = eigenvectors[:,j + 4 * k]
        function_as_vector = function_as_vector/np.max(function_as_vector)
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i - 1][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im1 = ax[i][j].scatter(X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ),  c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im1, ax = ax[i][j] )
    # end for
    ax[i][0].set_title( "For "+str(1 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][1].set_title( "For "+str(2 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][2].set_title( "For "+str(3 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][3].set_title( "For "+str(4 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    #ax[i][4].set_title("For $5^{th}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
    k += 1
# end for
#fig.colorbar(im1, ax=ax.ravel().tolist())
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/EigenheatmaplowestL1.pdf", format = 'pdf')
plt.show();

#### Heatmaps of eigenvectors corresponding to the 16 largest eigenvalues

In [ ]:
eigenmodes_count = 16
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:], y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = eigenmodes_count//4, ncols = eigenmodes_count//4, sharey = True )
scale = 30
courant_modes = []
k = 0
for i in range(eigenmodes_count//4):
    eigenvectors = eigvecs[1]
    courant_modes.append( [] )
    for j in range(eigenmodes_count//4):
        function_as_vector = eigenvectors[:, N[0] + N[1] - j - 1 - 4 * k]
        function_as_vector = function_as_vector/np.max( function_as_vector )
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i - 1][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation<-0.1:
                function_as_vector = - function_as_vector
            #print( "epsilon: ", i, " -- corr = ",
            #  correlation )
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im = ax[i][eigenmodes_count//4 - j - 1].scatter(X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ),  c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im, ax = ax[i][ eigenmodes_count//4 - j - 1 ])
    # end for
    ax[i][3].set_title( "For "+str(N[0] + N[1] - 1 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][2].set_title( "For "+str(N[0] + N[1] - 2 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][1].set_title( "For "+str(N[0] + N[1] - 3 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][0].set_title( "For "+str(N[0] + N[1] - 4 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    #ax[i][0].set_title("For $5^{th}$ largest eigenvalue $\epsilon$= "+str(epsilons[i]))
    k += 1
# plt.axis("off")
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )   
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/EigenheatmaplargestL1.pdf", format = 'pdf' )
plt.show();



## L4-norm

In [ ]:
# Damped Newton
print("Damped Newton... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
dampedNewtonP_L4 = []
results_dampedNewton_L4  = []
times_dampedNewton_L4    = []
Hessians_dampedNewton_L4 = []
#Cost matrix
C = p_norms( x, y, 4 )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
epsilons = [ 1.0, 0.5, 0.1, 0.05 ]
for eps in epsilons:
    # Line Search
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    f, g = 0 * a, 0 * b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton( K,
                                                a,
                                                b,
                                                f,
                                                g,
                                                eps,
                                                rho,
                                                c )
    out = Optimizer._update(    max_iterations = 50,
                                debug = False )
    end = time.time()
    if out != -1:
        results_dampedNewton_L4.append( out )
        times_dampedNewton_L4.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        dampedNewtonP_L4.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n" )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        Hessians_dampedNewton_L4.append( mat )
    else:
        epsilons.remove( eps )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewton_L4)):
  error = np.asarray( results_dampedNewton_L4[i]['error_a'] ) + np.asarray( results_dampedNewton_L4[i]['error_b'] ) 
  plt.plot( error, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/ErrorLinesearchNewtonL4.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

### Objective function plot

In [ ]:
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( "Objective Function" )
for i in range(len(results_dampedNewton_L4)):
  value = np.asarray(results_dampedNewton_L4[i]['objective_values'])
  plt.plot( value, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.yscale( 'log' )
plt.legend()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/ObjectiveLineSearchNewtonL4.pdf", format = 'pdf' )
plt.show()

### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title("Alpha")
for i in range(len(results_dampedNewton_L4)):
  plt.plot( np.asarray(results_dampedNewton_L4[i]["linesearch_steps"]), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/AlphaLineSearchNewtonL4.pdf", format = 'pdf' )
plt.show()

In [ ]:
eigs = []
eigvecs = []
for i in range(len(epsilons)):
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps)  )
    Hessian = Hessians_dampedNewton_L4[i]
    ev = print_spectral_statistics( Hessian, stabilize = False )
    eigs.append(ev[0])
    eigvecs.append(ev[1])
    print("")
# end for

In [ ]:
fig, ax = plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    if not np.isnan(eigs[i]).any():
        ax[i].hist( eigs[i], 50 )
        ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
        ax[i].set_xlabel( "Eigenvalues" )
        ax[i].set_yscale( "log" )
    else:
        print( "The eigenvalues for "+str(epsilons[i])+ " contains Nans." )
# end for
plt.subplots_adjust( wspace = 0, hspace = 0.5 )
plt.savefig( "../Images/Eigenvectorstability_analysis_images/eigenhistunstabilizedL4.pdf", format = 'pdf' )
plt.show()

### Heatmaps

#### Heatmaps of eigenvectors corresponding to the 4 smallest eigenvalues 

In [ ]:
eigenmodes_count = 4
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:],y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = len(epsilons), ncols = eigenmodes_count, sharey = True )
scale = 30
courant_modes = []
for i in range(len(epsilons)):
    eigenvectors = eigvecs[i]
    courant_modes.append( [] )
    for j in range(eigenmodes_count):
        function_as_vector = eigenvectors[:,j]
        function_as_vector = function_as_vector/np.max( function_as_vector )
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i - 1][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
            #print( "epsilon: ", i, " -- corr = ", correlation )
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im1 = ax[i][j].scatter(X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ),  c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im1, ax = ax[i][j] )
    # end for
    ax[i][0].set_title( "For smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][1].set_title( "For $2^{nd}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][2].set_title( "For $3^{rd}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][3].set_title( "For $4^{th}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    #ax[i][4].set_title("For $5^{th}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
# end for
#fig.colorbar(im1, ax=ax.ravel().tolist())
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/Eigenheatmaplowest4L4.pdf", format = 'pdf' )
plt.show();

#### Heatmaps of eigenvectors corresponding to the 16 smallest eigenvalues 

In [ ]:
eigenmodes_count = 16
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:], y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = eigenmodes_count//4, ncols = eigenmodes_count//4, sharey = True )
scale = 30
courant_modes = []
k = 0
for i in range(eigenmodes_count//4):
    eigenvectors = eigvecs[1]
    courant_modes.append( [] )
    for j in range(eigenmodes_count//4):
        function_as_vector = eigenvectors[:, j + 4 * k]
        function_as_vector = function_as_vector/np.max( function_as_vector )
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i -1 ][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im1 = ax[i][j].scatter(X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ),  c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im1, ax = ax[i][j] )
    # end for
    ax[i][0].set_title( "For "+str(1 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][1].set_title( "For "+str(2 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][2].set_title( "For "+str(3 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][3].set_title( "For "+str(4 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    #ax[i][4].set_title("For $5^{th}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
    k += 1
# end for
#fig.colorbar(im1, ax=ax.ravel().tolist())
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1)
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1)
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/EigenheatmaplowestL4.pdf", format = 'pdf' )
plt.show();

#### Heatmaps of eigenvectors corresponding to the 16 largest eigenvalues

In [ ]:
eigenmodes_count = 16
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:], y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize=( 20, 20 ), nrows = eigenmodes_count//4, ncols = eigenmodes_count//4, sharey = True )
scale = 30
courant_modes = []
k = 0
for i in range(eigenmodes_count//4):
    eigenvectors = eigvecs[1]
    courant_modes.append( [] )
    for j in range(eigenmodes_count//4):
        function_as_vector = eigenvectors[:, N[0] + N[1] - j - 1 - 4 * k]
        function_as_vector = function_as_vector/np.max( function_as_vector )
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i - 1][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
            #print( "epsilon: ", i, " -- corr = ",
            #  correlation )
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im = ax[i][eigenmodes_count//4 - j - 1].scatter(X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ),  c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im, ax=  ax[i][eigenmodes_count//4 - j - 1 ] )
    # end for
    ax[i][3].set_title( "For "+str(N[0] + N[1] - 1 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][2].set_title( "For "+str(N[0] + N[1] - 2 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][1].set_title( "For "+str(N[0] + N[1] - 3 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][0].set_title( "For "+str(N[0] + N[1] - 4 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    #ax[i][0].set_title("For $5^{th}$ largest eigenvalue $\epsilon$= "+str(epsilons[i]))
    k += 1
# plt.axis("off")
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )   
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/EigenheatmaplargestL4.pdf", format = 'pdf' )
plt.show();

## L-infinity

In [ ]:
# Damped Newton
print("Damped Newton... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
dampedNewtonP_Linfty = []
results_dampedNewton_Linfty  = []
times_dampedNewton_Linfty    = []
Hessians_dampedNewton_Linfty = []
#Cost matrix
C = p_norms( x, y, 'inf' )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones(N[1] ) )
epsilons = [ 1.0, 0.5, 0.1, 0.05,0.03 ]
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )  
    #Kernel
    K = np.exp( - C/eps )
    f, g = 0 * a, 0 * b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton( K,
                                                a,
                                                b,
                                                f,
                                                g,
                                                eps,
                                                rho,
                                                c )
    out = Optimizer._update(    max_iterations = 50,
                                debug = False )
    end = time.time()
    if out != -1:
        results_dampedNewton_Linfty.append( out )
        times_dampedNewton_Linfty.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        dampedNewtonP_Linfty.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n" )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        Hessians_dampedNewton_Linfty.append( mat )
    else:
        epsilons.remove( eps )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewton_Linfty)):
  error = np.asarray( results_dampedNewton_Linfty[i]['error_a'] ) + np.asarray( results_dampedNewton_Linfty[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/ErrorLinesearchNewtonLinfinity.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

### Objective function plot

In [ ]:
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( "Objective Function" )
for i in range(len(results_dampedNewton_Linfty)):
  value = np.asarray(results_dampedNewton_Linfty[i]['objective_values'])
  plt.plot( value, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.yscale( 'log' )
plt.legend()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/ObjectiveLineSearchNewtonLinfinity.pdf", format = 'pdf' )
plt.show()

### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Alpha" )
for i in range(len(results_dampedNewton_Linfty)):
  plt.plot( np.asarray( results_dampedNewton_Linfty[i]["linesearch_steps"] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" ) 
plt.ylabel( "Alpha in log-scale" )
plt.legend()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/AlphaLineSearchNewtonLinfinity.pdf", format = 'pdf' )
plt.show()

In [ ]:
eigs = []
eigvecs = []
for i in range(len(epsilons)):
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    Hessian = Hessians_dampedNewton_Linfty[i]
    ev = print_spectral_statistics( Hessian, stabilize = False )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
fig, ax = plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    if not np.isnan( eigs[i] ).any():
        ax[i].hist( eigs[i], 50 )
        ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
        ax[i].set_xlabel( "Eigenvalues" )
        ax[i].set_yscale( "log" )
    else:
        print( "The eigenvalues for "+str(epsilons[i])+ " contains Nans." )
# end for
plt.subplots_adjust( wspace = 0, hspace = 0.5 )
plt.savefig( "../Images/Eigenvectorstability_analysis_images/eigenhistunstabilizedLinfinity.pdf", format = 'pdf' )
plt.show()

### Heatmaps

#### Heatmaps of eigenvectors corresponding to the 4 smallest eigenvalues 

In [ ]:
eigenmodes_count = 4
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:], y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = len(epsilons), ncols = eigenmodes_count, sharey = True )
scale = 30
courant_modes = []
for i in range(len(epsilons)):
    eigenvectors = eigvecs[i]
    courant_modes.append( [] )
    for j in range(eigenmodes_count):
        function_as_vector = eigenvectors[:,j]
        function_as_vector = function_as_vector/np.max( function_as_vector )
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i-1][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
            #print( "epsilon: ", i, " -- corr = ", correlation )
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im1 = ax[i][j].scatter(X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ) , c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im1, ax = ax[i][j] )
    # end for
    ax[i][0].set_title( "For smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][1].set_title( "For $2^{nd}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][2].set_title( "For $3^{rd}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][3].set_title( "For $4^{th}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    #ax[i][4].set_title("For $5^{th}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
# end for
#fig.colorbar(im1, ax=ax.ravel().tolist())
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/Eigenheatmaplowest4Linf.pdf", format = 'pdf' )
plt.show();

#### Heatmaps of eigenvectors corresponding to the 16 smallest eigenvalues 

In [ ]:
eigenmodes_count = 16
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:], y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = eigenmodes_count//4, ncols = eigenmodes_count//4, sharey = True )
scale = 30
courant_modes = []
k = 0
for i in range(eigenmodes_count//4):
    eigenvectors = eigvecs[1]
    courant_modes.append( [] )
    for j in range(eigenmodes_count//4):
        function_as_vector = eigenvectors[:, j + 4 * k]
        function_as_vector = function_as_vector/np.max( function_as_vector )
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i - 1][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im1 = ax[i][j].scatter(X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ),  c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im1, ax= ax[i][j] )
    # end for
    ax[i][0].set_title( "For "+str(1 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][1].set_title( "For "+str(2 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][2].set_title( "For "+str(3 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][3].set_title( "For "+str(4 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    #ax[i][4].set_title("For $5^{th}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
    k += 1
# end for
#fig.colorbar(im1, ax=ax.ravel().tolist())
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/EigenheatmaplowestLinfinity.pdf", format = 'pdf' )
plt.show();

#### Heatmaps of eigenvectors corresponding to the 16 largest eigenvalues

In [ ]:
eigenmodes_count = 16
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:], y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = eigenmodes_count//4, ncols = eigenmodes_count//4, sharey = True )
scale = 30
courant_modes = []
k = 0
for i in range(eigenmodes_count//4):
    eigenvectors = eigvecs[1]
    courant_modes.append( [] )
    for j in range(eigenmodes_count//4):
        function_as_vector = eigenvectors[:, N[0] + N[1] - j - 1 - 4 * k]
        function_as_vector = function_as_vector/np.max( function_as_vector )
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i - 1][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
            #print( "epsilon: ", i, " -- corr = ",
            #  correlation )
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im = ax[i][eigenmodes_count//4 - j - 1].scatter(X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ),  c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im, ax = ax[i][eigenmodes_count//4 - j - 1] )
    # end for
    ax[i][3].set_title( "For "+str(N[0] + N[1] - 1 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][2].set_title( "For "+str(N[0] + N[1] - 2 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][1].set_title( "For "+str(N[0] + N[1] - 3 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][0].set_title( "For "+str(N[0] + N[1] - 4 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    #ax[i][0].set_title("For $5^{th}$ largest eigenvalue $\epsilon$= "+str(epsilons[i]))
    k += 1
# plt.axis("off")
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )   
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/EigenheatmaplargestLinfinity.pdf", format = 'pdf' )
plt.show();

## 1-cosine similarity

In [ ]:
# Damped Newton
print("Damped Newton... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
dampedNewtonP_cosine = []
results_dampedNewton_cosine  = []
times_dampedNewton_cosine   = []
Hessians_dampedNewton_cosine = []
#Cost matrix
C = 1 - ( x.transpose().dot( y ) )/( np.linalg.norm( x, 2 ) * np.linalg.norm( y, 2 ) )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
epsilons = [ 1.0, 0.5, 0.1, 0.05, 0.03 ]
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    f, g = 0 * a, 0 * b
    print( " |- Iterating")  
    start = time.time()
    Optimizer = computational_OT.damped_Newton( K,
                                                a,
                                                b,
                                                f,
                                                g,
                                                eps,
                                                rho,
                                                c )
    out = Optimizer._update(    max_iterations = 50,
                                debug = False )
    end = time.time()
    if out != -1:
        results_dampedNewton_cosine.append( out )
        times_dampedNewton_cosine.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        dampedNewtonP_cosine.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n" )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        Hessians_dampedNewton_cosine.append( mat )
    else:
        epsilons.remove( eps )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range(len(results_dampedNewton_cosine)):
  error = np.asarray( results_dampedNewton_cosine[i]['error_a'] ) + np.asarray( results_dampedNewton_cosine[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/ErrorLinesearchNewtoncosine.pdf", format = 'pdf' )
plt.show()
print("\n Error plots can increase! The error is not the objective function!")

### Objective function plot

In [ ]:
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( "Objective Function" )
for i in range(len(results_dampedNewton_cosine)):
  value = np.asarray( results_dampedNewton_cosine[i]['objective_values'] )
  plt.plot( value, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2)
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.yscale( 'log' )
plt.legend()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/ObjectiveLineSearchNewtoncosine.pdf", format = 'pdf' )
plt.show()

### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title("Alpha")
for i in range(len(results_dampedNewton_cosine)):
  plt.plot( np.asarray( results_dampedNewton_cosine[i]["linesearch_steps"] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/AlphaLineSearchNewtoncosine.pdf", format = 'pdf' )
plt.show()

In [ ]:
eigs = []
eigvecs = []
for i in range(len(epsilons)):
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    Hessian = Hessians_dampedNewton_cosine[i]
    ev = print_spectral_statistics( Hessian, stabilize = False )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print( "" )
# end for

In [ ]:
fig, ax = plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    if not np.isnan(eigs[i]).any():
        ax[i].hist( eigs[i], 50 )
        ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
        ax[i].set_xlabel( "Eigenvalues" )
        ax[i].set_yscale( "log" )
    else:
        print( "The eigenvalues for "+str(epsilons[i])+ " contains Nans." )
# end for
plt.subplots_adjust( wspace = 0, hspace = 0 )
plt.savefig( "../Images/Eigenvectorstability_analysis_images/eigenhistunstabilizedcosine.pdf", format = 'pdf' )
plt.show()

### Heatmaps

#### Heatmaps of eigenvectors corresponding to the 4 smallest eigenvalues 

In [ ]:
eigenmodes_count = 4
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:], y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = len(epsilons), ncols = eigenmodes_count, sharey = True )
scale = 30
courant_modes = []
for i in range(len(epsilons)):
    eigenvectors = eigvecs[i]
    courant_modes.append( [] )
    for j in range(eigenmodes_count):
        function_as_vector = eigenvectors[:,j]
        function_as_vector = function_as_vector/np.max( function_as_vector )
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i - 1][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
            #print( "epsilon: ", i, " -- corr = ", correlation )
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im1 = ax[i][j].scatter(X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ),  c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im1, ax = ax[i][j] )
    # end for
    ax[i][0].set_title( "For smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][1].set_title( "For $2^{nd}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][2].set_title( "For $3^{rd}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    ax[i][3].set_title( "For $4^{th}$ smallest eigenvalue $\epsilon$ = "+str(epsilons[i]) )
    #ax[i][4].set_title("For $5^{th}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
# end for
#fig.colorbar(im1, ax=ax.ravel().tolist())
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/Eigenheatmaplowest4LCosineSim.pdf", format = 'pdf' )
plt.show();

#### Heatmaps of eigenvectors corresponding to the 16 smallest eigenvalues 

In [ ]:
eigenmodes_count = 16
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:], y[1,:] ) )[:,None]  ))
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = eigenmodes_count//4, ncols = eigenmodes_count//4, sharey=  True )
scale = 30
courant_modes = []
k = 0
for i in range(eigenmodes_count//4):
    eigenvectors = eigvecs[1]
    courant_modes.append( [] )
    for j in range(eigenmodes_count//4):
        function_as_vector = eigenvectors[:, j + 4 * k]
        function_as_vector = function_as_vector/np.max( function_as_vector )
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i - 1][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im1 = ax[i][j].scatter(X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ),  c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar( im1, ax = ax[i][j] )
    # end for
    ax[i][0].set_title( "For "+str(1 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][1].set_title( "For "+str(2 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][2].set_title( "For "+str(3 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][3].set_title( "For "+str(4 + 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    #ax[i][4].set_title("For $5^{th}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
    k += 1
# end for
#fig.colorbar(im1, ax=ax.ravel().tolist())
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout()
plt.savefig( "../Images/Eigenvectorstability_analysis_images/Eigenheatmaplowestcosine.pdf", format = 'pdf' )
plt.show();

#### Heatmaps of eigenvectors corresponding to the 16 largest eigenvalues

In [ ]:
eigenmodes_count = 16
X = np.hstack( ( np.hstack( ( x[0,:], y[0,:] ) )[:,None], np.hstack( ( x[1,:], y[1,:] ) )[:,None] ) )
fig, ax = plt.subplots( figsize = ( 20, 20 ), nrows = eigenmodes_count//4, ncols = eigenmodes_count//4, sharey = True )
scale = 30
courant_modes = []
k = 0
for i in range(eigenmodes_count//4):
    eigenvectors = eigvecs[1]
    courant_modes.append( [] )
    for j in range(eigenmodes_count//4):
        function_as_vector = eigenvectors[:, N[0] + N[1] - j - 1 - 4 * k]
        function_as_vector = function_as_vector/np.max( function_as_vector )
        # Flip sign if anti-correlated
        if i > 0:
            old_mode = courant_modes[i - 1][j]
            correlation = np.dot( old_mode, function_as_vector )/( np.linalg.norm( old_mode ) * np.linalg.norm( function_as_vector ) )
            if correlation < - 0.1:
                function_as_vector = - function_as_vector
            #print( "epsilon: ", i, " -- corr = ",
            #  correlation )
        # Record
        courant_modes[i].append( function_as_vector )
        # Plot
        im = ax[i][eigenmodes_count//4 -j-1].scatter(X.T[0,:], X.T[1,:], s = scale, edgecolors = ( 0, 0, 0, 0 ),  c = function_as_vector, cmap = 'plasma', linewidths = 2 )
        fig.colorbar(im, ax = ax[i] [eigenmodes_count//4 - j - 1 ] )
    # end for
    ax[i][3].set_title( "For "+str(N[0] + N[1] - 1 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][2].set_title( "For "+str(N[0] + N[1] - 2 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][1].set_title( "For "+str(N[0] + N[1] - 3 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    ax[i][0].set_title( "For "+str(N[0] + N[1] - 4 - 4 * k)+"th  eigenvalue $\epsilon$ = "+str(epsilons[1]) )
    #ax[i][0].set_title("For $5^{th}$ largest eigenvalue $\epsilon$= "+str(epsilons[i]))
    k += 1
# plt.axis("off")
plt.xlim( np.min( X.T[0,:] ) - .1, np.max( X.T[0,:] ) + .1 )   
plt.ylim( np.min( X.T[1,:] ) - .1, np.max( X.T[1,:] ) + .1 )
plt.subplots_adjust( wspace = 0, hspace = 0.09 )
plt.tight_layout() 
plt.savefig( "../Images/Eigenvectorstability_analysis_images/Eigenheatmaplargestcosine.pdf", format = 'pdf' )
plt.show();